In [ ]:
!pip install -q torchaudio omegaconf soundfile

In [3]:
import torch
import zipfile
import torchaudio
from glob import glob
from itertools import groupby

device = torch.device('cpu')  # gpu also works, but our models are fast enough for CPU

model, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en', # also available 'de', 'es'
                                       device=device)
(read_batch, split_into_batches,
 read_audio, prepare_model_input) = utils  # see function signature for details

# download a single file, any format compatible with TorchAudio (soundfile backend)
torch.hub.download_url_to_file('https://opus-codec.org/static/examples/samples/speech_orig.wav',
                               dst ='speech_orig.wav', progress=True)
test_files = glob('speech_orig.wav')
batches = split_into_batches(test_files, batch_size=10)
input = prepare_model_input(read_batch(batches[0]),
                            device=device)
print("input shape",input.shape)
output = model(input)
print(output.shape)
for example in output:
    x=example.cpu()
    print(x.shape)
    print(decoder(x))

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


  0%|          | 0.00/0.99M [00:00<?, ?B/s]

input shape torch.Size([1, 172800])
torch.Size([1, 136, 999])
torch.Size([136, 999])
the boch canoe slit on the smooth planks blew the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us


In [6]:
from torch.utils.mobile_optimizer import optimize_for_mobile

# quantized_model = torch.quantization.quantize_dynamic(
#     model, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8)
scripted_model = torch.jit.script(model)
optimized_model = optimize_for_mobile(scripted_model)
optimized_model._save_for_lite_interpreter("silero.ptl")

In [10]:
model=torch.jit.load('/content/silero.ptl')

output = model(input)
print(output.shape)
for example in output:
    x=example.cpu()
    print(x.shape)
    print(decoder(x))

torch.Size([1, 136, 999])
torch.Size([136, 999])
the boch canoe slit on the smooth planks blew the sheet to the dark blue background it's easy to tell a depth of a well four hours of steady work faced us
